# FCN
Trains the fully convolutional Network used to create the foreground mask.  
Network is based on U-Net

### Location of images:
    config.IMAGE_PATH
### XML file used to label the images (labeling done with CVAT and labes exported as '')
    config.ANNOTATION_FILE_FCN
### Batchsize used during the training of the FCN
    config.BATCH_SIZE_FCN = 10
### Epoches of the Training
    config.EPOCHES_FCN = 50

Final model is saved in ../models/mask.pth


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from Training.FCN.label import Label
from Training.FCN.dataLoader import MaskDataset
from Training.FCN.unet import UNet


import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as pltimage
from PIL import Image
import config

from torch import optim
import torch.nn as nn
import torchvision.transforms as transforms
import torch


In [ ]:
data = Label(config.ANNOTATION_FILE_FCN, 0.3, (256, 256))

In [ ]:
h = data.annotations[6]
print(len(data.annotations))
test = h['mask']
plt.imshow(test)
plt.show


In [ ]:
c = Image.open(config.IMAGE_PATH + "/" + h['file'])
plt.imshow(c)
plt.show

In [ ]:
if config.DEVICE == 'cuda':
    print('Loading Model on GPU')
    cnn = UNet(n_channels=3, n_classes=1, bilinear=False).cuda()
else:
    print('Loading Model on CPU')
    cnn = UNet(n_channels=3, n_classes=1, bilinear=False)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(cnn.parameters(),lr = 0.001 )

counter = []
loss_history = [] 
iteration_number= 0

dataset = MaskDataset(data.annotations, transforms.Compose([transforms.Resize([256, 256]), transforms.ToTensor()]))


train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                               batch_size=config.BATCH_SIZE_FCN,
                                               shuffle=False)

first =True
for epoch in range(0,config.EPOCHES_FCN):
    print('EPOCH:', epoch)
    z = 0
    for i, (img1_set, label, _) in enumerate(train_loader):
        # Display first image
        if first:
            plt.imshow(img1_set[0].swapaxes(0,1).swapaxes(1,2))
            plt.show()
            plt.imshow(label[0][0])
            plt.show()
            first = False
        z = z + 1
        
        # load images to GPU
        if config.DEVICE == "cuda":
            img1_set = img1_set.cuda()
            label = label.cuda()

        optimizer.zero_grad()
        # Feed Forward
        output1 = cnn(img1_set)
        # calculate Loss
        loss = criterion(output1, label.float())
        
        #back propagation
        loss.backward()
        
        optimizer.step()
        
        # Show progress every 10 Batches
        if z %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss.item()))
            iteration_number +=10 * config.BATCH_SIZE_FCN
            counter.append(iteration_number)
            loss_history.append(loss.item())
        

In [ ]:
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

show_plot(counter,loss_history)

In [ ]:
# Save Model
torch.save(cnn.state_dict(), '../models/mask.pth')